In [0]:


import json

# Retrieve the bronze_params directly as a widget
bronze_params = dbutils.widgets.get("bronze_params")
print(f"Raw bronze_params: {bronze_params}")

# Parse the JSON string
output_data = json.loads(bronze_params)

# Access individual variables
start_date = output_data.get("start_date", "")
end_date = output_data.get("end_date", "")
bronze_adls = output_data.get("bronze_adls", "")
silver_adls = output_data.get("silver_adls", "")
gold_adls = output_data.get("gold_adls", "")

print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")


In [0]:
# Step 1: Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime
from pyspark.sql.types import DoubleType, LongType
from pyspark.sql.functions import col, explode

spark = SparkSession.builder.getOrCreate()

# Step 2: Define paths
bronze_path = f"{bronze_adls}earthquake_data_{start_date}.json"
silver_path = f"{silver_adls}earthquake_data_2025-12-21/"

# Step 3: Load raw JSON into Spark DataFrame
df = spark.read.option("multiLine", True).json(bronze_path)



In [0]:
df.head()

Row(bbox=[-155.50883483887, -61.9834, -1.6, 161.4175, 64.857, 432.202], features=[Row(geometry=Row(coordinates=[-101.069, 32.145, 8.3252], type='Point'), id='tx2025zbqkld', properties=Row(alert=None, cdi=None, code='2025zbqkld', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2025zbqkld&format=geojson', dmin=0.2, felt=None, gap=82, ids=',tx2025zbqkld,', mag=1.4, magType='ml', mmi=None, net='tx', nst=17, place='23 km SSW of Westbrook, Texas', rms=0.1, sig=30, sources=',tx,', status='reviewed', time=1766360903266, title='M 1.4 - 23 km SSW of Westbrook, Texas', tsunami=0, type='earthquake', types=',origin,phase-data,', tz=None, updated=1766421849295, url='https://earthquake.usgs.gov/earthquakes/eventpage/tx2025zbqkld'), type='Feature'), Row(geometry=Row(coordinates=[-104.341, 31.664, 6.7664], type='Point'), id='tx2025zbqbzw', properties=Row(alert=None, cdi=None, code='2025zbqbzw', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2025zbqbzw&format=geojs

In [0]:
df=df.limit(100)

In [0]:
# Step 4: Reshape / flatten nested JSON
df_exploded = df.select(explode(col("features")).alias("feature"))
df = (
    df_exploded
    .select(
        col("feature.id").alias("id"),
        col("feature.geometry.coordinates").getItem(0).alias("longitude"),
        col("feature.geometry.coordinates").getItem(1).alias("latitude"),
        col("feature.geometry.coordinates").getItem(2).alias("elevation"),
        col("feature.properties.title").alias("title"),
        col("feature.properties.place").alias("place_description"),
        col("feature.properties.sig").alias("sig"),
        col("feature.properties.mag").alias("mag"),
        col("feature.properties.magType").alias("magType"),
        col("feature.properties.time").alias("time"),
        col("feature.properties.updated").alias("updated")
    )
)



In [0]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: long (nullable = true)
 |-- updated: long (nullable = true)



In [0]:
# Step 5: Convert data types
df = df.withColumn("longitude", col("longitude").cast(DoubleType())) \
       .withColumn("latitude", col("latitude").cast(DoubleType())) \
       .withColumn("elevation", col("elevation").cast(DoubleType())) \
       .withColumn("mag", col("mag").cast(DoubleType())) \
       .withColumn("time", from_unixtime(col("time")/1000).cast("timestamp")) \
       .withColumn("updated", from_unixtime(col("updated")/1000).cast("timestamp"))

# Step 6: Filter out rows with null longitude, latitude, or time
df = df.filter(
    col("longitude").isNotNull() &
    col("latitude").isNotNull() &
    col("time").isNotNull()
)



In [0]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)



In [0]:
# Step 7: Write to Silver container in Parquet format
df.write.mode("append").parquet(silver_path)

print(f"Data successfully written to Silver container: {silver_path}")


Data successfully written to Silver container: abfss://silver@earthquakedata.dfs.core.windows.net/earthquake_data_2025-12-21/


In [0]:
df.head(1)

[Row(id='tx2025zbqkld', longitude=-101.069, latitude=32.145, elevation=8.3252, title='M 1.4 - 23 km SSW of Westbrook, Texas', place_description='23 km SSW of Westbrook, Texas', sig=30, mag=1.4, magType='ml', time=datetime.datetime(2025, 12, 21, 23, 48, 23), updated=datetime.datetime(2025, 12, 22, 16, 44, 9))]

In [0]:
dbutils.notebook.exit(silver_path)